<a href="https://colab.research.google.com/github/shreyashkar-ml/Pytorch-learning/blob/main/min_char_rnn_explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with open('/content/input.txt', 'r') as f:
  data = f.read()

In [ ]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f"data has {data_size} characters, {vocab_size} unique.")

data has 1115394 characters, 65 unique.


In [ ]:
# Each character in the vocabulary gets a unique integer index assigned, in the
# half-open interval [0:N). These indices are useful to create one-hot encoded
# vectors to represent characters in numerical computations.
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

print('char_to_ix', char_to_ix)
print('ix_to_char', ix_to_char)

char_to_ix {'w': 0, 'O': 1, 'i': 2, 'M': 3, 'n': 4, 'd': 5, 'T': 6, 'q': 7, 'Q': 8, '3': 9, 'o': 10, 'l': 11, 'J': 12, 'L': 13, '.': 14, 'U': 15, ';': 16, 'G': 17, 'p': 18, "'": 19, 'f': 20, '!': 21, 'z': 22, 'b': 23, '$': 24, 'x': 25, 'A': 26, 'H': 27, 'W': 28, 'm': 29, 'r': 30, 'a': 31, ':': 32, 'V': 33, ',': 34, 'g': 35, 'D': 36, 'P': 37, 'X': 38, 'F': 39, 'e': 40, 'j': 41, 'h': 42, 'I': 43, '\n': 44, 'C': 45, 'Y': 46, 't': 47, '&': 48, 'R': 49, 'B': 50, '?': 51, ' ': 52, 's': 53, 'E': 54, 'k': 55, 'v': 56, '-': 57, 'y': 58, 'S': 59, 'K': 60, 'c': 61, 'u': 62, 'N': 63, 'Z': 64}
ix_to_char {0: 'w', 1: 'O', 2: 'i', 3: 'M', 4: 'n', 5: 'd', 6: 'T', 7: 'q', 8: 'Q', 9: '3', 10: 'o', 11: 'l', 12: 'J', 13: 'L', 14: '.', 15: 'U', 16: ';', 17: 'G', 18: 'p', 19: "'", 20: 'f', 21: '!', 22: 'z', 23: 'b', 24: '$', 25: 'x', 26: 'A', 27: 'H', 28: 'W', 29: 'm', 30: 'r', 31: 'a', 32: ':', 33: 'V', 34: ',', 35: 'g', 36: 'D', 37: 'P', 38: 'X', 39: 'F', 40: 'e', 41: 'j', 42: 'h', 43: 'I', 44: '\n', 45: 

In [ ]:
# Hyperparameters for RNN
hidden_size = 100       # size of hidden layer of neurons
seq_length = 16         # number of steps to unroll the RNN for
learning_rate = 1e-1

In [ ]:
# Stop when processed this much data
MAX_DATA = 1000000

In [ ]:
# Model parameters/weights -- these are shared among all steps.
# Weights initialized randomly; biases initialized to 0.
# Inputs are characters one-hot encoded in a vocab-sized vector.
# Dimensions: H = hidden_size, V = vocab_size
Wxh = np.random.randn(hidden_size, vocab_size) * 0.01       # input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01      # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01       # hidden to output
bh = np.zeros((hidden_size, 1))                 # hidden bias
by = np.zeros((vocab_size, 1))                  # output bias

## **Overview of RNN**
Recurrent Neural Networks are at the core an attempt to develop an internal structure that is appropriate for a particular task domain using internal 'hidden' units which are not part of the input or output vectors.

Learning becomes more interesting but more difficult when we introduce hidden units whose actual desired states are not specified by the task. The simplest for of the learning procedure is for layered networks which have a layer of inputs at the bottom; any number of intermediate layers; and a layer of output units at the top.

An input vector is presented to the network by setting the states of the input units.

Then the stats of the units in each layer are determined by applying steps as followed for input vector $ x_t $ :
- **Hidden State Calculation:**
</br> $ h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h) $

- **Output and Softmax:**
<br> $ y_t = W_{hy} \cdot h_t + b_y $
<br> $ p_t = \frac{ \exp(y_t) }{ \sum exp(y_t) } $ </br>

where, $ h_{t-1} $ represents the hidden state input from previous states, $ b_h $ represents the biased term in hidden state calculation, and $ p_t $ represents the softmax output from the output vector $ y_t $.

RNNs are particularly effective in tasks like language modeling, machine translation, etc. where the context of previous characters are crucial for predicting the next one.


Now, let's work through min-char-rnn code one-step at a time

### `lossFun` Function:
This function runs both forward and backward passes through the RNN and computes the loss and gradients.

```python
def lossFun(inputs, targets, hprev):
  """
  Runs forward and backward passes through the RNN.

  inputs, targets: Lists of integers. For some i, inputs[i] is the input
                   character (encoded as an index to the ix_to_char map) and
                   targets[i] is the corresponding next character in the
                   training data (similarly encoded).
  hprev: Hx1 array of initial hidden state.
  returns: loss, gradients on model parameters, and last hidden state.
  """
```
**Inputs**:
- `inputs`: Indices representing the input characters.
- `targets`: Indices representing the next characters in the sequence
- `hprev`: Initial hidden state from the previous sequence

**Outputs**:
- `loss`: Cross-entropy loss
- Gradients for the weights and biases (`dWxh, dWhh, dWhy, dbh, dby`)
- The last hidden state (`hs[len(inputs)-1]`)



#### Forward Pass

The forward pass computes the hidden states and the outputs at each time step.

```python
# Initialize storage for variables needed for forward and backward passes
xs, hs, ys, ps = {}, {}, {}, {}
hs[-1] = np.copy(hprev) # Initialize with the given hidden state
loss = 0
```
For each time step $ t $:
1. **Input Encoding**: The input characters are converted into one-hot encoding vectors for input into the model.
```python
xs[t] = np.zeros((vocab_size, 1)) # one-hot encoding
xs[t][inputs[t]] = 1
```
2. **Hidden State Calculation**: The hidden states to capture the task information and develop/emulate an internal structure is computed using:
$$ h_t = \tanh(W_{xh}\cdot x_t + W_{hh} \cdot h_{t-1} + b_h) $$
```python
hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
```
3. **Output and Softmax**: We first calculate the unnormalized scores (`ys[t]`) and then converet those into softmax probabilities (`ps[t]`) for output:
$$ y_t = W_{hy} \cdot h_t + b_y $$
$$ p_t = \frac{\exp(y_t)}{\sum \exp(y_t)} $$
```python
ys[t] = np.dot(Why, hs[t]) + by
ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
```
4. **Loss Calculation**: The cross-entropy loss at each time step is then added up using:
```python
loss += -np.log(ps[t][targets[t],0])
```



#### Backpropagation through time (BPTT)

1. **Gradient of Loss w.r.t Softmax Output (`ps[t]`):**
For each time step t:
The loss at time step $ t $ is given by:
$$ Loss_t = - \log(p_{t,target}) $$
The derivative of the loss w.r.t softmax probabilities $p_t$ is:
$$ \frac{\partial Loss_t}{\partial p_t} = p_t - 1_{target} $$
Where:
- $ p_t $ is the softmax probability vector at time step t.
- $ 1_{target} $ is a one-hot vector with a 1 at the index of the target character.
```python
dy = np.copy(ps[t])
dy[targets[t]] -= 1
```

2. **Gradient w.r.t Output Weights (`Why`) and Bias (`by`):**
The output $ y_t $ at each time step is computed as:
$$ y_t = W_{hy} \cdot h_t + b_y $$
The gradients of the loss w.r.t the weights and biases are given by:
$$ \frac{ \partial Loss_t }{\partial W_{hy}} =
\sum_t \left(\frac{ \partial Loss_t }{ \partial y_t } \cdot \frac{ \partial y_t }{ \partial W_{hy}} \right)= \sum_t( p_t - 1_{target}).h_t^T $$
$$ \frac {\partial Loss_t }{\partial b_y} = \sum_t(p_t - 1_{target}) $$
```python
dWhy += np.dot(dy, hs[t].T)
dby += dy
```

3. **Gradient w.r.t Hidden State (`h_t`):**
To backpropagate into the hidden state, we need to account for both the current time step's gradient and the incoming gradient from the next time step:
$$ \frac{ \partial Loss_t }{\partial h_t} = W_{hy}^T \cdot \frac{\partial Loss_t}{ \partial y_t} + \frac{\partial Loss_{t+1}}{\partial h_t} $$
Where:
- $ \frac{\partial Loss_{t+1}}{\partial h_t} $ is the gradient passed back from the next time step.
```python
dh = np.dot(Why.T, dy) + dhnext
```

4. **Gradient w.r.t Activation Function (`tanh`):**
The hidden state is computed using the $ tanh $ activation function:
$$ h_t = \tanh(W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h) $$
The input to the `tanh` activation function is:
$$ a_t = W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h $$
The gradient through the $ tanh $ function is:
$$ dhraw = \frac{\partial Loss_t}{\partial a_t} = \frac{ \partial Loss_t }{\partial ({W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h })} = (1 - h_{t}^2) \odot \frac{ \partial Loss_t }{\partial h_t} $$
Here, $ (1-h_t^2) $ is the derivative of $ tanh(h_t) $.
```python
dhraw = (1- hs[t]*hs[t]) * dh
```


5. **Gradient w.r.t Input Weights (`Wxh`), Hidden Weights (`Whh`), and Hidden Bias (`bh`):**
Now, compute the gradients w.r.t weights and biases connecting the inputs and hidden states:
The input to the `tanh` activation function is:
$$ a_t = W_{xh} \cdot x_t + W_{hh} \cdot h_{t-1} + b_h $$
- For input-to-hidden weights $ W_{xh} $:
$$
\frac{ \partial \text{Loss}_t}{\partial W_{xh}} = \sum_t \left( \frac{ \partial \text{Loss}_t}{\partial a_t} \cdot \frac{ \partial a_t}{\partial W_{xh}} \right)= \sum_t \left( dhraw \cdot x_t^T \right)
$$
- For hidden-to-hidden weights $ W_{hh} $:
$$
\frac{ \partial \text{Loss}_t}{\partial W_{hh}} = \sum_t \left(\frac{ \partial \text{Loss}_t}{\partial a_t} \cdot \frac{ \partial a_t }{\partial W_{hh}} \right) = \sum_t (dhraw \cdot h_{t-1}^T)
$$
- For hidden bias $ b_h $:
$$
\frac{\partial Loss_t}{\partial b_h} = \sum_t \frac{\partial Loss}{\partial a_t} = \sum_tdhraw
$$
```python
dbh += dhraw
dWxh += np.dot(dhraw, xs[t].T)
dWhh += np.dot(dhraw, hs[t-1].T)
```

6. **Gradient Propagation to Previous Time Step (`dhnext`):**
Propogate the gradient back to the previous time step:
$$
\frac{ \partial Loss_t}{\partial h_{t-1}} = W_{hh}^T \cdot \frac{\partial Loss_t}{\partial a_t} = W_{hh}^T \cdot dhraw
$$
```python
dhnext = np.dot(Whh.T, dhraw)
```

##### Summary:
- **Step 1:** Compute gradient of the loss w.r.t the output probabilities.
- **Step 2:** Calculate the gradients for the weights and biases connecting hidden states to outputs.
- **Step 3:** Propagate the gradient through the hidden state, taking into account the contribution from the next time step.
- **Step 4:** Backpropagate through the $ tanh $ activation function.
- **Step 5:** Compute the gradients w.r.t the weights and biases connecting the inputs and hidden states, as well as the hidden-to-hidden weights.
- **Step 6:** Propagate the gradient back to the previous time step's hidden state.

These steps iteratively update the gradient accumulations by iterating backward through the time steps of the sequene.

In [ ]:
def lossFun(inputs, targets, hprev):
  """
  Runs forward and backward passes through the RNN.

  inputs, targets: Lists of integers. For some i, inputs[i] is the input
                   character (encoded as an index to the ix_to_char map) and
                   targets[i] is the corresponding next character in the
                   training data (similarly encoded).
  hprev: Hx1 array of initial hidden state.
  returns: loss, gradients on model parameters, and last hidden state.
  """
  # Caches that keep values computed in the forward pass at each time step,
  # to be reused in the backward pass.
  xs, hs, ys, ps = {}, {}, {}, {}

  # Initial incoming state
  hs[-1] = np.copy(hprev)
  loss = 0

  # Forward pass
  for t in range(len(inputs)):
    # Input at time step t is xs[t].  Prepare a one-hot encoded vector of shape
    # (V, 1). inputs[t] is the index where the 1 goes.
    xs[t] = np.zeros((vocab_size, 1))       # encode 1-of-k representation
    xs[t][inputs[t]] = 1

    # Compute h[t] from h[t-1] and x[t]
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)

    # Compute ps[t] - softmax probabilities for output
    ys[t] = np.dot(Why, hs[t]) + by
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))

  loss += -np.log(ps[t][targets[t], 0])

  # Backward pass: compute gradients going backwards
  # Gradients are initialized to 0s, and every time step contributes to them
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)

  # Initialize the incoming gradient of h to zero; this is a safe assumption for
  # a sufficiently long unrolling.
  dhnext = np.zeros_like(hs[0])

  # The backwards pass iterates over the input sequence backwards.
  for t in reversed(range(len(inputs))):
    # Backprop through the gradients of loss and softmax
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1

    # Compute gradients for the Why and by parameters
    dWhy += np.dot(dy, hs[t].T)
    dby += dy

    # Backprop through the fully-connected layer (Why, by) to h. Also add up the
    # incoming gradient for h from the next cell.
    # Note: proper Jacobian matmul here would be dy.dot(Why), that would give
    # a [1,T] vector. Since we need [T,1] for h, we flip the dot (we could have
    # transposed after everything, too)
    dh = np.dot(Why.T, dy) + dhnext

    # Backprop through tanh
    dhraw = (1 - hs[t]*hs[t]) * dh

    # Compute gradients for the dby, dWxh, Whh parameters.
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)

    # Backprop the gradient to the incoming h, which will be used in the previous time step.
    dhnext = np.dot(Whh.T, dhraw)

  # Gradient clipping to the range [-5, 5]
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out = dparam)

  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [ ]:
def sample(h, seed_ix, n):
  """
  Sample a sequence of integeres from the model.

  Runs the RNN in forward mode for n steps;
  seed_ix --> The seed letter for the first time step,
  h --> The memory state.

  Returns a sequence of letters produced by the model (indices).
  """
  # Create a one-hot vector to represent the input.
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1

  ixes = []    # Initializes an empty list ixes to store the generated character indices

  for t in range(n):
    # Run the forward pass only.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))

    # Sample from the distribution produced by softmax.
    ix = np.random.choice(range(vocab_size), p = p.ravel())

    # Prepare input for the next call
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [ ]:
# Gradient checking
from random import uniform

def gradCheck(inputs, targets, hprev):
  """
  Performs gradient checking to verify that the analytical gradients computed by backpropagation
  match the numerical gradients approximated by finite differences.
  """

  global Wxh, Whh, Why, bh, by   # Declare global variables for the RNN parameters.

  # Set the number of checks and the small value for numerical gradient calculation.
  num_checks, delta = 10, 10e-5

  # Compute the analytical gradients using the backpropagation function 'lossFun'
  _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFun(inputs, targets, hprev)
  for param, dparam, name in zip([Wxh, Whh, Why, bh, by],
                                 [dWxh, dWhh, dWhy, dbh, dby],
                                 ['Wxh', 'Whh', 'Why', 'bh', 'by']):
    # Verify that the dimensions of the parameter and its gradient match.
    s0 = dparam.shape
    s1 = param.shape
    assert s0 == s1, 'Error dims dont match: %s and %s.' % (s0, s1)
    print(name)

    for i in range(num_checks):

      # randomly select an index 'ri' in the flattened parameter array to check.
      ri = int(uniform(0, param.size))
      # evaluate cost at [x + delta] and [x - delta]

      old_val = param.flat[ri]

      # compute loss with a positive perturbation at index 'ri'
      param.flat[ri] = old_val + delta
      cg0, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)  # cg0 is the cost with positive perturbation

      # compute loss with a negative perturbation at index 'ri'
      param.flat[ri] = old_val - delta
      cg1, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)  # cg1 is the cost with negative perturbation

      param.flat[ri] = old_val    # reset old value for this parameter


      grad_analytic = dparam.flat[ri]   # analytical gradient

      grad_numerical = (cg0 - cg1) / (2 * delta) # numerical gradient

      rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)

      print('%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error))
      # rel_error should be on order of 10e-7 or less


In [ ]:
# This function invokes gradCheck with all the parameters properly set up.
def basicGradCheck():
  inputs = [char_to_ix[ch] for ch in data[:seq_length]]
  targets = [char_to_ix[ch] for ch in data[1:seq_length + 1]]
  hprev = np.zeros((hidden_size, 1))      # reset RNN memory
  gradCheck(inputs, targets, hprev)

In [ ]:
basicGradCheck()

Wxh
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000022, -0.010993 => 1.004099e+00 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
0.000000, 0.000000 => nan 
-0.000000, 0.002755 => 1.000000e+00 
0.000000, 0.000000 => nan 
Whh
0.000286, -0.000009 => 1.064862e+00 
0.000037, -0.000186 => 1.498958e+00 
-0.000175, -0.000089 => 3.236123e-01 
0.000051, -0.000144 => 2.089605e+00 
-0.000001, -0.000248 => 9.942125e-01 
-0.000019, -0.000058 => 5.032077e-01 
0.000070, 0.000087 => 1.114804e-01 
0.000246, 0.000126 => 3.212826e-01 
-0.000015, 0.000427 => 1.072026e+00 
0.000098, 0.000241 => 4.226525e-01 
Why
0.000139, -0.000088 => 4.379527e+00 
0.000055, 0.000650 => 8.441306e-01 
-0.000169, -0.000024 => 7.513294e-01 
-0.000256, 0.001042 => 1.651875e+00 
-0.000037, -0.000500 => 8.625015e-01 
-0.000194, -0.000241 => 1.080471e-01 
-0.000030, 0.007120 => 1.008393e+00 
-0.000024, -0.000072 => 5.043514e-01 
-0.000022, 0.000276 => 1.1729

In [ ]:
# n is the iteration counter; p is the sequence pointer, at the beginning
# of each step it points at the sequence in the input that will be used for
# training this iteration.
n, p = 0, 0

# Memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)
smooth_loss = -np.log(1.0 / vocab_size)*seq_length

while p < MAX_DATA:
  # Preprare inputs (we're sweeping from left to right in steps seq_length long)
  if (p+seq_length+1) >= len(data) or n == 0:
    hprev = np.zeros((hidden_size, 1))      # reset RNN memory
    p = 0                                   # go from start of data

  # In each step we unroll the RNN for seq_length cells, and present it with
  # seq_length inputs and seq_length target outputs to learn.

  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # Sample from the model now and then
  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # Forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 200 == 0: print('iter %d (p=%d), loss: %f' % (n, p, smooth_loss))

  # Perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)

  p += seq_length
  n += 1

----
 jItHb
&&P:iwF-;o?yJIcbfmtr,;-':FQktiUYyjc UqqLLaOI&vWL3cX;upkDYXcT-Xo'jTMISLSkSKuxFirUUUDm?EX-rENrubNr,ihp$tQrUE;'IGkjcDDbuvI;TP:mNK!KoUEJlU-pHGdk'Fku3X-GTE.,QuIzLjIBjnuhu?UiSZAWGE:.Upfgv,j&BT$uCg;:pB 
----
iter 0 (p=0), loss: 66.727580
iter 200 (p=3200), loss: 55.333248
iter 400 (p=6400), loss: 45.886402
iter 600 (p=9600), loss: 38.130851
iter 800 (p=12800), loss: 31.752094
----
 pir t hor ony: l:is ma
Toud h aondet
Vieus orl tom.
L ongf antassulos Ve.y,,

VpUIe I
sUGo de 3y mhh: the thiw Led oh henon aeuwsaens
V a de

OrGSg ctayau lu fwin,, s auu di thetrhonbaai atasenuesalIl 
----
iter 1000 (p=16000), loss: 26.498397
iter 1200 (p=19200), loss: 22.180755
iter 1400 (p=22400), loss: 18.639481
iter 1600 (p=25600), loss: 15.764989
iter 1800 (p=28800), loss: 13.387624
----
 n baseu nhphetthho wcisbe lnithen, tathetel,
Wot darsisi waras olp desbea apeth nut'er harikathvz shaceese Moil orcon gonudheo mow
Aesu ne vebt thegen bid on me odpl bes-arelles kayyo., 

Mgul yoldeI  
----
iter 